<a href="https://colab.research.google.com/github/shubhangickore/Python_practice/blob/main/pr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello world")

hello world


In [ ]:
a =10
if a>5 :
      print('true')

true


In [ ]:
import pandas as pd
import numpy as np

def compute_max_employees_during_tenure(df):
    """
    df has columns: employee_id, hire_date (datetime), termination_date (datetime or NaT)
    Returns DataFrame with employee_id, max_employees, first_date_of_max
    """
    # 1. Prepare events list
    events = []
    for _, row in df.iterrows():
        emp = row['employee_id']
        h = row['hire_date']
        t = row['termination_date']
        # Hire event
        events.append((h, +1, emp))
        # Termination event: we do not include the termination date as working day
        if pd.notnull(t):
            # event at t with -1
            events.append((t, -1, emp))

    # 2. Create DataFrame of events
    events_df = pd.DataFrame(events, columns=['date', 'delta', 'employee_id'])
    # Sort by date, and for same date, apply +1 before -1 (so hires counted before fires)
    events_df = events_df.sort_values(by=['date', 'delta'], ascending=[True, False])
    events_df = events_df.reset_index(drop=True)

    # 3. Compute running count of employees
    events_df['running_total'] = events_df['delta'].cumsum()

    # Also keep track of for each event, what the running_total is
    # Now we want, for each date, how many employees were working just after that event

    # 4. For each employee, filter the slice of events between their hire_date (inclusive)
    #    and termination_date (exclusive), find the maximum running_total and first date
    results = []
    for _, row in df.iterrows():
        emp = row['employee_id']
        h = row['hire_date']
        t = row['termination_date']
        # Filter events in [h, t) — if t is NaT, then take all events >= h
        if pd.isnull(t):
            slice_ev = events_df[events_df['date'] >= h]
        else:
            slice_ev = events_df[(events_df['date'] >= h) & (events_df['date'] < t)]

        if slice_ev.empty:
            # If somehow no events in that period, fallback
            results.append({'employee_id': emp, 'max_employees': 0, 'first_date_of_max': h})
        else:
            # find the max running_total
            max_val = slice_ev['running_total'].max()
            # find first date where running_total == max_val
            first_date = slice_ev[slice_ev['running_total'] == max_val]['date'].iloc[0]
            results.append({'employee_id': emp, 'max_employees': int(max_val), 'first_date_of_max': first_date})

    return pd.DataFrame(results)

# Example usage:
data = {
    'employee_id': [1,2,3],
    'hire_date': pd.to_datetime(['2018-01-10', '2019-03-20', '2020-01-05']),
    'termination_date': pd.to_datetime(['2020-06-15', None, '2021-12-01'])
}
df = pd.DataFrame(data)
out = compute_max_employees_during_tenure(df)
print(out)


   employee_id  max_employees first_date_of_max
0            1              3        2020-01-05
1            2              3        2020-01-05
2            3              3        2020-01-05


In [ ]:
import pandas as pd
df = pd.DataFrame({
    'city':["pune","mumbai","pune","bangalore"],
    'rides':[15,20,5,33]
})

df.groupby('city')['rides'].sum()

,rides
city,
bangalore,33
mumbai,20
pune,20


In [5]:
from scipy.stats import ttest_ind

group1 = [33, 11, 22, 44]
group2 = [21, 31, 41, 51]

t_stat,p_val = ttest_ind(group1,group2)
print(t_stat,p_val)


-0.8857851797221404 0.4098264210613309
